In [51]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import talib as ta
from talib import abstract
import yfinance as yfin
from arch import arch_model
from walk_forward_module import walk_forward


pd.set_option('display.float_format', '{:.2f}'.format)


## 資料DataFrame

In [52]:
df = yfin.download('^TWII', start = '2011-1-7', end = '2017-12-30')
df.columns = ['open', 'high', 'low', 'close', 'adj_close', 'volume']

[*********************100%%**********************]  1 of 1 completed


# function

In [53]:
def get_llt(df, a):
    llt = np.zeros(len(df))
    llt[0] = df['close'].iloc[0]
    llt[1] = (1 - a) * df['close'].iloc[0] + a * df['close'].iloc[1]
    for n in range(2, len(df)):
        llt[n] = ((a - (a ** 2)/4) * df['close'].iloc[n]) + (((a ** 2)/2) * df['close'].iloc[n - 1]) - ((a - (3 * (a ** 2))/4) * df['close'].iloc[n - 2]) + ((2 * (1 - a)) * llt[n - 1]) - (((1 - a) ** 2) * llt[n - 2])
    
    df['llt'] = llt
    df['llt_slope'] = np.nan

    for i in range(0, len(df)):
        df.loc[df.index[i], 'llt_slope'] = df['llt'].iloc[i] - df['llt'].iloc[i - 1]
    return df


def bolinger_band(df, p, q, o, rolling, num_vol, upper_threshold, lower_threshold, retrace_u_threshold, retrace_l_threshold):
    model = arch_model(df['llt'].dropna(), vol = 'Garch', p = p, q = q, o = o, rescale = False)
    garch_fit = model.fit(disp = 'off')
    df['garch_volitility'] = garch_fit.conditional_volatility

    df['bolinger_upper'] = df['llt'] + num_vol * df['garch_volitility']
    df['bolinger_lower'] = df['llt'] - num_vol * df['garch_volitility']

    df['bolinger_upper_smooth'] = df['bolinger_upper'].ewm(span = 20).mean()
    df['bolinger_lower_smooth'] = df['bolinger_lower'].ewm(span = 20).mean()

    df['upper_max'] = df['bolinger_upper'].rolling(window = rolling).max()
    df['lower_min'] = df['bolinger_lower'].rolling(window = rolling).min()

    df['upper_retrace'] = (df['upper_max'] - df['bolinger_upper']) / df['upper_max']
    df['lower_retrace'] = (df['bolinger_lower'] - df['lower_min']) / df['lower_min']

    df['upper_deviation'] = (df['bolinger_upper'] - df['close']) / df['close']
    df['lower_deviation'] = (df['close'] - df['bolinger_lower']) / df['close']

    df['upper_rise'] = df['upper_deviation'] >= upper_threshold
    df['lower_down'] = df['lower_deviation'] >= lower_threshold
    df['upper_down'] = df['upper_retrace'] >= retrace_u_threshold
    df['lower_rise'] = df['lower_retrace'] >= retrace_l_threshold

    return df

def get_middle(df):
    
    df['middle'] = np.nan
    df.loc[df.index[0], 'middle'] = df['close'].iloc[0]

    for i in range(1, len(df)):

        upper_change = abs(df['bolinger_upper_smooth'].iloc[i] - df['bolinger_upper_smooth'].iloc[i - 1])
        lower_change = abs(df['bolinger_lower_smooth'].iloc[i] - df['bolinger_lower_smooth'].iloc[i - 1])
        
        peak_upward = upper_change > lower_change
        peak_downward = upper_change < lower_change
    
        if peak_upward:
            df.iloc[i, df.columns.get_loc('middle')] = ((df['middle'].iloc[i - 1] * i) + df['bolinger_lower_smooth'].iloc[i]) / (i + 1)
            # df.loc[df.index[i], 'middle'] = ((df['middle'].iloc[i - 1] * (i)) + df['bolinger_lower']) / (i + 1)
        elif peak_downward:
            df.iloc[i, df.columns.get_loc('middle')] = ((df['middle'].iloc[i - 1] * i) + df['bolinger_upper_smooth'].iloc[i]) / (i + 1)
            # df.loc[df.index[i], 'middle'] = ((df['middle'].iloc[i - 1] * (i)) + df['bolinger_upper']) / (i + 1)
    
    return df



# strategy

In [54]:
def llt_strategy(df, fund, feePaid, a, p, q, o, rolling, num_vol, upper_threshold, lower_threshold, retrace_u_threshold, retrace_l_threshold):
    
    ret_df = pd.DataFrame(index = df.index)
    equity = pd.DataFrame(index = df.index)
    
    df = get_llt(df, a)
    df = bolinger_band(
        df = df, p = p, q = q, o = o, rolling = rolling, num_vol = num_vol,
        upper_threshold = upper_threshold, lower_threshold = lower_threshold,
        retrace_u_threshold = retrace_u_threshold, retrace_l_threshold = retrace_l_threshold)
    df = get_middle(df)

    BS = None
    t = 0
    df['Bull'] = False
    df['Bear'] = False
    ret_df = ret_df.assign(
        position = np.zeros(len(df)),
        buy_price = np.zeros(len(df)),
        sell_price = np.zeros(len(df)),
        buy = np.zeros(len(df)),
        sell = np.zeros(len(df)),
        sellshort = np.zeros(len(df)),
        buytocover = np.zeros(len(df)),
        buy_time = np.nan,
        sell_time = np.nan,
        sellshort_time = np.nan,
        buytocover_time = np.nan,
        short_price = np.zeros(len(df)),
        buytocover_price = np.zeros(len(df)),
        profit_list = np.zeros(len(df)),
        profit_fee_list = np.zeros(len(df)),
        profit_fee_list_realized = np.zeros(len(df))
    )
    ret_df['buy_time'] = pd.to_datetime(ret_df['buy_time'], errors='coerce')
    ret_df['sell_time'] = pd.to_datetime(ret_df['sell_time'], errors='coerce')
    ret_df['sellshort_time'] = pd.to_datetime(ret_df['sellshort_time'], errors='coerce')
    ret_df['buytocover_time'] = pd.to_datetime(ret_df['buytocover_time'], errors='coerce')

    for i in range(2, len(df) - 1):
        
        condition1 = (df['bolinger_upper_smooth'].iloc[i] - df['middle'].iloc[i]) > (df['middle'].iloc[i] - df['bolinger_lower_smooth'].iloc[i]) # 峰向上
        condition2 = (df['bolinger_upper_smooth'].iloc[i] - df['middle'].iloc[i]) < (df['middle'].iloc[i] - df['bolinger_lower_smooth'].iloc[i]) # 峰向下
        condition3 = (df['bolinger_upper_smooth'].iloc[i] - df['bolinger_upper_smooth'].iloc[i - 1]) > 0 # 上軌向上
        condition4 = (df['bolinger_upper_smooth'].iloc[i] - df['bolinger_upper_smooth'].iloc[i - 1]) < 0 # 上軌向下
        condition5 = (df['bolinger_lower_smooth'].iloc[i] - df['bolinger_lower_smooth'].iloc[i - 1]) > 0 # 下軌向上
        condition6 = (df['bolinger_lower_smooth'].iloc[i] - df['bolinger_lower_smooth'].iloc[i - 1]) < 0 # 下軌向下
        df.at[df.index[i], 'Bull'] = (condition1 & condition3) | (condition2 & condition5)
        df.at[df.index[i], 'Bear'] = (condition1 & condition4) | (condition2 & condition6)

        # 進出場邏輯
        entryLong = (df['llt_slope'].iloc[i] > 0) & (((df['upper_rise'].iloc[i] == True) & condition1) | ((df['lower_rise'].iloc[i] == True) & condition2))
        exitLong = ((df['bolinger_upper'].iloc[i] < df['bolinger_upper_smooth'].iloc[i]) & condition1 & condition4) | ((df['bolinger_lower'].iloc[i] < df['bolinger_lower_smooth'].iloc[i]) & condition2 & condition6)
        entryShort = (df['llt_slope'].iloc[i] < 0) & (((df['upper_down'].iloc[i] == True) & condition1) | ((df['lower_down'].iloc[i] == True) & condition2))
        exitShort = ((df['bolinger_upper'].iloc[i] > df['bolinger_upper_smooth'].iloc[i]) & condition1 & condition3) | ((df['bolinger_lower'].iloc[i] > df['bolinger_lower_smooth'].iloc[i]) & condition2 & condition5)

        if BS == None:
            if df['Bull'].iloc[i]:
                if entryLong:
                    BS = 'B'
                    t = i + 1
                    ret_df.at[ret_df.index[t], 'buy'] = t
                    ret_df.at[ret_df.index[t], 'buy_price'] = df['open'].iloc[t]
                    ret_df.at[ret_df.index[t], 'buy_time'] = df.index[t]
                    ret_df.at[ret_df.index[t], 'position'] += 1

            elif df['Bear'].iloc[i]:
                if entryShort:
                    BS = 'S'
                    t = i + 1
                    ret_df.at[ret_df.index[t], 'sellshort'] = t
                    ret_df.at[ret_df.index[t], 'short_price'] = df['open'].iloc[t]
                    ret_df.at[ret_df.index[t], 'sellshort_time'] = df.index[t]
                    ret_df.at[ret_df.index[t], 'position'] -= 1

        elif BS == 'B':
            
            ret_df.at[ret_df.index[i + 1], 'position'] = ret_df.at[ret_df.index[i], 'position']
            profit = 200 * (df['open'].iloc[i + 1] - df['open'].iloc[i]) * ret_df['position'].iloc[i + 1]
            ret_df.at[ret_df.index[i], 'profit_list'] = profit

            if exitLong:
                pl_round = 200 * (df['open'].iloc[i + 1] - df['open'].iloc[t]) * ret_df['position'].iloc[i]
                profit_fee = profit - feePaid * 2
                ret_df.at[ret_df.index[i + 1], 'position'] -= 1
                ret_df.at[ret_df.index[i], 'profit_fee_list'] = profit_fee
                ret_df.at[ret_df.index[i + 1], 'sell_price'] = df['open'].iloc[i + 1]
                ret_df.at[ret_df.index[i + 1], 'sell_time'] = df.index[i + 1]
                ret_df.at[ret_df.index[i + 1], 'sell'] = i + 1
                BS = None

                ret_df.at[ret_df.index[i + 1], 'profit_fee_list_realized'] = pl_round - feePaid * 2

            elif i == len(df) - 2:
                if ret_df['position'].iloc[len(df) - 2] != 0:
                    unit = ret_df['position'].iloc[len(df) - 2]
                    profit_fee = profit - feePaid * 2
                    pl_round = 200 * (df['open'].iloc[i + 1] - df['open'].iloc[t]) * ret_df['position'].iloc[i]
                    ret_df.at[ret_df.index[i + 1], 'position'] -= unit
                    ret_df.at[ret_df.index[i], 'profit_fee_list'] = profit_fee
                    ret_df.at[ret_df.index[i + 1], 'sell_price'] = df['open'].iloc[i + 1]
                    ret_df.at[ret_df.index[i + 1], 'sell_time'] = df.index[i + 1]
                    ret_df.at[ret_df.index[i + 1], 'sell'] = i + 1
                    BS = None

            else:
                profit_fee = profit
                ret_df.at[ret_df.index[i], 'profit_fee_list'] = profit_fee

        elif BS == 'S':
            ret_df.at[ret_df.index[i + 1], 'position'] = ret_df.at[ret_df.index[i], 'position']
            profit = 200 * (df['open'].iloc[i + 1] - df['open'].iloc[i]) * ret_df['position'].iloc[i + 1]
            ret_df.at[ret_df.index[i], 'profit_list'] = profit

            if exitShort:
                pl_round = 200 * (df['open'].iloc[i + 1] - df['open'].iloc[t]) * ret_df['position'].iloc[i]
                profit_fee = profit - feePaid * 2
                ret_df.at[ret_df.index[i + 1], 'position'] += 1
                ret_df.at[ret_df.index[i], 'profit_fee_list'] = profit_fee
                ret_df.at[ret_df.index[i + 1], 'buytocover_price'] = df['open'].iloc[i + 1]
                ret_df.at[ret_df.index[i + 1], 'buytocover_time'] = df.index[i + 1]
                ret_df.at[ret_df.index[i + 1], 'buytocover'] = i + 1
                BS = None

                profit_fee_realized = pl_round - feePaid * 2
                ret_df.at[ret_df.index[i + 1], 'profit_fee_list_realized'] = profit_fee_realized

            elif i == len(df) - 2:
                if ret_df['position'].iloc[len(df) - 2] != 0:
                    unit = ret_df['position'].iloc[len(df) - 2]
                    profit_fee = profit - feePaid * 2
                    pl_round = 200 * (df['open'].iloc[i + 1] - df['open'].iloc[t]) * ret_df['position'].iloc[i]
                    ret_df.at[ret_df.index[i + 1], 'position'] += unit
                    ret_df.at[ret_df.index[i], 'profit_fee_list'] = profit_fee
                    ret_df.at[ret_df.index[i + 1], 'sell_price'] = df['open'].iloc[i + 1]
                    ret_df.at[ret_df.index[i + 1], 'buytocover_time'] = df.index[i + 1]
                    ret_df.at[ret_df.index[i + 1], 'buytocover'] = i + 1
                    BS = None
            else:
                profit_fee = profit
                ret_df.at[ret_df.index[i], 'profit_fee_list'] = profit_fee

        else:
            print('error')
            
        if t is not None:
            if pd.notna((ret_df['buy_time'].iloc[t]) and pd.notna(ret_df['sell_time'].iloc[i + 1])):
                hold_duration = (ret_df['sell_time'].iloc[i + 1] - ret_df['buy_time'].iloc[t]).days
                ret_df.at[ret_df.index[i + 1], 'hold_duration'] = hold_duration
            elif pd.notna((ret_df['sellshort_time'].iloc[t]) and pd.notna(ret_df['buytocover_time'].iloc[i + 1])):
                hold_duration = (ret_df['sellshort_time'].iloc[i + 1] - ret_df['buytocover_time'].iloc[t]).days
                ret_df.at[ret_df.index[i + 1], 'hold_duration'] = hold_duration
        else:
            print(f"Warning: t is not assigned before using it at index {i}")

        # print(f'Index: {i}, df["Bull"].iloc[i]: {df["Bull"].iloc[i]}, entryLong: {entryLong}, entryShort: {entryShort}')


    df['strategy_ret'] = ret_df['profit_list'].cumsum()
    equity['profitfee'] = ret_df['profit_fee_list'].cumsum()

    return ret_df, equity

# calculate metrics

In [55]:
def calculate_metrics(ret_df, equity, fund, risk_free_rate = 0.04/252):
    duration = (equity.index[-1] - equity.index[0]).days
    equity['equity'] = equity['profitfee'] + fund
    equity['drawdown_percent'] = (equity['equity']/equity['equity'].cummax()) - 1
    equity['drawdown'] = equity['equity'] - equity['equity'].cummax()
    profit = equity['profitfee'].iloc[-1]
    ret = equity['equity'].iloc[-1]/equity['equity'].iloc[0] - 1
    mdd = abs(equity['drawdown_percent'].min())
    calmarRatio = ret / mdd if mdd != 0 else None
    tradeTimes = min((ret_df['buy'] > 0).sum(), (ret_df['sell'] > 0).sum()) + min((ret_df['sellshort'] > 0).sum(), (ret_df['buytocover'] > 0).sum())
    winRate = len([i for i in ret_df['profit_fee_list_realized'] if i > 0]) / tradeTimes if tradeTimes != 0 else None
    try:
        profitFactor = sum([i for i in ret_df['profit_fee_list_realized'] if i>0]) / abs(sum([i for i in ret_df['profit_fee_list_realized'] if i < 0]))
    except:
        profitFactor = None
    mean_ret = ret_df['profit_list'].mean()
    std_ret = ret_df['profit_list'].std()
    sharp = (mean_ret - risk_free_rate) / std_ret if std_ret != 0 else None

    metrics = {
        'duration' :  duration,
        'profit': profit,
        'return': ret,
        'max_drawdown': mdd,
        'caimar_ratio': calmarRatio,
        'trade_times': tradeTimes,
        'win_rate': winRate,
        'profit_factor': profitFactor,
        'sharp_ratio': sharp
    }

    return metrics

# optimize

In [56]:
def optimize(a_range, o_range, upper_threshold_range, lower_threshold_range, retrace_u_threshold_range, retrace_l_threshold_range, df, fund, feePaid, p = 1, q = 1, rolling = 20, num_vol = 1):
    
    results = []

    for a in a_range:
        for o in o_range:
            for upper_threshold in upper_threshold_range:
                for lower_threshold in lower_threshold_range:
                    for retrace_u_threshold in retrace_u_threshold_range:
                        for retrace_l_threshold in retrace_l_threshold_range:
                            df_copy = df.copy()

                            ret_df, equity = llt_strategy(
                                df_copy, fund, feePaid = feePaid, a = a, p = p, q = q, o = o, rolling = rolling, num_vol = num_vol,
                                upper_threshold = upper_threshold, lower_threshold = lower_threshold,
                                retrace_u_threshold = retrace_u_threshold, retrace_l_threshold = retrace_l_threshold
                            )

                            metrics = calculate_metrics(ret_df, equity, fund)

                            result = {
                                'a': a,
                                'p': p,
                                'q': q,
                                'o': o,
                                'num_vol': num_vol,
                                'rolling': rolling,
                                'upper_threshold': upper_threshold,
                                'lower_threshold': lower_threshold,
                                'retrace_u_threshold': retrace_u_threshold,
                                'retrace_l_threshold': retrace_l_threshold,
                                'duration' :  metrics['duration'],
                                'profit': metrics['profit'],
                                'return': metrics['return'],
                                'max_drawDown': metrics['max_drawdown'],
                                'caimar_ratio': metrics['caimar_ratio'],
                                'trade_times': metrics['trade_times'],
                                'win_rate': metrics['win_rate'],
                                'profit_factor': metrics['profit_factor'],
                                'sharp_ratio': metrics['sharp_ratio']
                            }
                            results.append(result)
                            print(f'a: {a}, o: {o}, upper_threshold: {upper_threshold}, lower_threshold: {lower_threshold}, retrace_u_threshold: {retrace_u_threshold}, retrace_l_threshold: {retrace_l_threshold}')

    results_df = pd.DataFrame(results)
    print("Results DataFrame:")
    print(results_df)
    print("Columns:", results_df.columns)
    return results_df

# parametors

In [57]:
a = np.arange(0.1, 1, 0.1)
p = 1
q = 1
# o = np.arange(0, 0.1, 0.01)
o = np.arange(0, 0.1, 0.1)
num_vol = 1
rolling = 20
# upper_threshold = np.arange(0.01, 0.1, 0.01)
# lower_threshold = np.arange(0.01, 0.1, 0.01)
# retrace_u_threshold = np.arange(0.01, 0.1, 0.01)
# retrace_l_threshold = np.arange(0.01, 0.1, 0.01)
upper_threshold = np.arange(0.01, 0.02, 0.01)
lower_threshold = np.arange(0.01, 0.02, 0.01)
retrace_u_threshold = np.arange(0.01, 0.02, 0.01)
retrace_l_threshold = np.arange(0.01, 0.02, 0.01)
fund = 1000000
feePaid = 600

# execute

In [58]:
results_df = optimize(
    a_range = a, o_range = o,
    upper_threshold_range = upper_threshold, lower_threshold_range = lower_threshold,
    retrace_u_threshold_range = retrace_u_threshold, retrace_l_threshold_range = retrace_l_threshold,
    df = df, fund = fund, feePaid = feePaid, p = p, q = q, rolling = rolling, num_vol = num_vol)

a: 0.1, o: 0.0, upper_threshold: 0.01, lower_threshold: 0.01, retrace_u_threshold: 0.01, retrace_l_threshold: 0.01
a: 0.2, o: 0.0, upper_threshold: 0.01, lower_threshold: 0.01, retrace_u_threshold: 0.01, retrace_l_threshold: 0.01
a: 0.30000000000000004, o: 0.0, upper_threshold: 0.01, lower_threshold: 0.01, retrace_u_threshold: 0.01, retrace_l_threshold: 0.01
a: 0.4, o: 0.0, upper_threshold: 0.01, lower_threshold: 0.01, retrace_u_threshold: 0.01, retrace_l_threshold: 0.01
a: 0.5, o: 0.0, upper_threshold: 0.01, lower_threshold: 0.01, retrace_u_threshold: 0.01, retrace_l_threshold: 0.01
a: 0.6, o: 0.0, upper_threshold: 0.01, lower_threshold: 0.01, retrace_u_threshold: 0.01, retrace_l_threshold: 0.01
a: 0.7000000000000001, o: 0.0, upper_threshold: 0.01, lower_threshold: 0.01, retrace_u_threshold: 0.01, retrace_l_threshold: 0.01
a: 0.8, o: 0.0, upper_threshold: 0.01, lower_threshold: 0.01, retrace_u_threshold: 0.01, retrace_l_threshold: 0.01
a: 0.9, o: 0.0, upper_threshold: 0.01, lower_thre

## extract

In [59]:
def extract_best_params(results_df, optimize_indicator, ascending = False):
    sorted_df = results_df.sort_values(by = optimize_indicator, ascending = ascending)
    best_params = sorted_df.iloc[0].to_dict()
    return best_params

In [60]:
results_df

,a,p,q,o,num_vol,rolling,upper_threshold,lower_threshold,retrace_u_threshold,retrace_l_threshold,duration,profit,return,max_drawDown,caimar_ratio,trade_times,win_rate,profit_factor,sharp_ratio
0,0.10,1,1,0.00,1,20,0.01,0.01,0.01,0.01,2548,-277593.55,-0.28,0.50,-0.56,62,0.34,0.83,-0.01
1,0.20,1,1,0.00,1,20,0.01,0.01,0.01,0.01,2548,36338.57,0.04,0.31,0.12,82,0.34,1.02,0.01
2,0.30,1,1,0.00,1,20,0.01,0.01,0.01,0.01,2548,-267005.76,-0.27,0.44,-0.61,105,0.37,0.86,-0.01
3,0.40,1,1,0.00,1,20,0.01,0.01,0.01,0.01,2548,-278154.79,-0.28,0.45,-0.61,118,0.36,0.86,-0.01
4,0.50,1,1,0.00,1,20,0.01,0.01,0.01,0.01,2548,-390620.80,-0.39,0.46,-0.84,126,0.33,0.81,-0.01
5,0.60,1,1,0.00,1,20,0.01,0.01,0.01,0.01,2548,-300921.09,-0.30,0.38,-0.80,126,0.33,0.85,-0.01
6,0.70,1,1,0.00,1,20,0.01,0.01,0.01,0.01,2548,-460243.36,-0.46,0.50,-0.92,131,0.31,0.79,-0.01
7,0.80,1,1,0.00,1,20,0.01,0.01,0.01,0.01,2548,-376851.76,-0.38,0.44,-0.86,133,0.32,0.81,-0.01
8,0.90,1,1,0.00,1,20,0.01,0.01,0.01,0.01,2548,-212397.85,-0.21,0.35,-0.60,138,0.32,0.89,-0.00


# walk forward optimize

In [61]:
# 执行 walk forward 回测
best_params = extract_best_params(results_df = results_df, optimize_indicator = 'sharp_ratio')

stats_master, params_results_df = walk_forward(
    strategy = llt_strategy,
    data_full = df,
    warmup_bars = 20,
    lookback_bars = 60,
    validation_bars = 20,
    fund = fund,
    feePaid = feePaid,
    optimize_fn = optimize,
    **best_params
)

Processing index: 80
Sample data:
              open    high     low   close  adj_close   volume
Date                                                          
2011-02-14 8666.61 8715.97 8626.66 8685.47    8685.43  3074000
2011-02-15 8679.12 8753.05 8565.05 8721.93    8721.89  3209400
2011-02-16 8740.32 8767.09 8702.81 8712.96    8712.92  2959800
2011-02-17 8738.71 8745.65 8669.77 8683.88    8683.84  3118400
2011-02-18 8763.69 8874.50 8763.69 8843.84    8843.80  3326600
Strategy parameters before optimization: {'a': 0.2, 'p': 1.0, 'q': 1.0, 'o': 0.0, 'num_vol': 1.0, 'rolling': 20.0, 'upper_threshold': 0.01, 'lower_threshold': 0.01, 'retrace_u_threshold': 0.01, 'retrace_l_threshold': 0.01, 'duration': 2548.0, 'profit': 36338.57421875, 'return': 0.036338574218750086, 'max_drawDown': 0.30638485288662665, 'caimar_ratio': 0.11860434312069813, 'trade_times': 82.0, 'win_rate': 0.34146341463414637, 'profit_factor': 1.0197583546878541, 'sharp_ratio': 0.005535358759085396}
Optimization failed wi

/var/folders/0j/sdf7gd116kd5lv12gdjc81900000gn/T/ipykernel_25318/1189408131.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['llt'] = llt
/var/folders/0j/sdf7gd116kd5lv12gdjc81900000gn/T/ipykernel_25318/1189408131.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['llt_slope'] = np.nan
/var/folders/0j/sdf7gd116kd5lv12gdjc81900000gn/T/ipykernel_25318/1189408131.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] =

Optimization failed with error: 
Strategy execution failed with error: 
Processing index: 560
Sample data:
              open    high     low   close  adj_close   volume
Date                                                          
2013-01-17 7748.57 7774.97 7603.27 7616.64    7616.60  2929200
2013-01-18 7697.28 7732.87 7676.11 7732.87    7732.83  2113400
2013-01-21 7725.36 7739.37 7674.18 7724.92    7724.88  1515400
2013-01-22 7746.79 7759.47 7699.25 7759.10    7759.06  1580400
2013-01-23 7773.20 7773.20 7730.10 7744.18    7744.14  1655600
Strategy parameters before optimization: {'a': 0.2, 'p': 1.0, 'q': 1.0, 'o': 0.0, 'num_vol': 1.0, 'rolling': 20.0, 'upper_threshold': 0.01, 'lower_threshold': 0.01, 'retrace_u_threshold': 0.01, 'retrace_l_threshold': 0.01, 'duration': 2548.0, 'profit': 36338.57421875, 'return': 0.036338574218750086, 'max_drawDown': 0.30638485288662665, 'caimar_ratio': 0.11860434312069813, 'trade_times': 82.0, 'win_rate': 0.34146341463414637, 'profit_factor': 1.0197

/var/folders/0j/sdf7gd116kd5lv12gdjc81900000gn/T/ipykernel_25318/1189408131.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['llt'] = llt
/var/folders/0j/sdf7gd116kd5lv12gdjc81900000gn/T/ipykernel_25318/1189408131.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['llt_slope'] = np.nan
/var/folders/0j/sdf7gd116kd5lv12gdjc81900000gn/T/ipykernel_25318/1189408131.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] =

Optimization failed with error: 
Strategy execution failed with error: 
Processing index: 1060
Sample data:
              open    high     low   close  adj_close   volume
Date                                                          
2015-01-30 9452.85 9466.65 9361.91 9361.91    9361.87  2041700
2015-02-02 9368.83 9410.43 9352.19 9386.99    9386.95  1650800
2015-02-03 9394.47 9460.17 9394.47 9448.73    9448.69  1649300
2015-02-04 9477.52 9533.88 9477.52 9513.92    9513.88  2134800
2015-02-05 9511.06 9517.42 9472.42 9512.05    9512.01  1523200
Strategy parameters before optimization: {'a': 0.2, 'p': 1.0, 'q': 1.0, 'o': 0.0, 'num_vol': 1.0, 'rolling': 20.0, 'upper_threshold': 0.01, 'lower_threshold': 0.01, 'retrace_u_threshold': 0.01, 'retrace_l_threshold': 0.01, 'duration': 2548.0, 'profit': 36338.57421875, 'return': 0.036338574218750086, 'max_drawDown': 0.30638485288662665, 'caimar_ratio': 0.11860434312069813, 'trade_times': 82.0, 'win_rate': 0.34146341463414637, 'profit_factor': 1.019

/var/folders/0j/sdf7gd116kd5lv12gdjc81900000gn/T/ipykernel_25318/1189408131.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['llt'] = llt
/var/folders/0j/sdf7gd116kd5lv12gdjc81900000gn/T/ipykernel_25318/1189408131.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['llt_slope'] = np.nan
/var/folders/0j/sdf7gd116kd5lv12gdjc81900000gn/T/ipykernel_25318/1189408131.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] =

Optimization failed with error: 
Strategy execution failed with error: 
Processing index: 1560
Sample data:
              open    high     low   close  adj_close   volume
Date                                                          
2017-03-01 9751.12 9758.78 9674.78 9674.78    9674.77  2474000
2017-03-02 9727.60 9774.60 9646.34 9691.80    9691.79  2522300
2017-03-03 9672.26 9679.82 9633.88 9648.21    9648.20  2180100
2017-03-06 9647.00 9688.50 9647.00 9682.63    9682.62  1906000
2017-03-07 9690.26 9738.07 9690.20 9738.07    9738.06  1882200
Strategy parameters before optimization: {'a': 0.2, 'p': 1.0, 'q': 1.0, 'o': 0.0, 'num_vol': 1.0, 'rolling': 20.0, 'upper_threshold': 0.01, 'lower_threshold': 0.01, 'retrace_u_threshold': 0.01, 'retrace_l_threshold': 0.01, 'duration': 2548.0, 'profit': 36338.57421875, 'return': 0.036338574218750086, 'max_drawDown': 0.30638485288662665, 'caimar_ratio': 0.11860434312069813, 'trade_times': 82.0, 'win_rate': 0.34146341463414637, 'profit_factor': 1.019

/var/folders/0j/sdf7gd116kd5lv12gdjc81900000gn/T/ipykernel_25318/1189408131.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['llt'] = llt
/var/folders/0j/sdf7gd116kd5lv12gdjc81900000gn/T/ipykernel_25318/1189408131.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['llt_slope'] = np.nan
/var/folders/0j/sdf7gd116kd5lv12gdjc81900000gn/T/ipykernel_25318/1189408131.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] =

In [62]:
stats_master

[]